In [1]:
import torch
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt

In [2]:
# Training data
train = datasets.FashionMNIST("", train=True, download=True, transform=transforms.Compose([transforms.ToTensor()]))

# Testing data
test = datasets.FashionMNIST("", train=False, download=True, transform=transforms.Compose([transforms.ToTensor()]))

# Loading the datasets
trainset = torch.utils.data.DataLoader(train, batch_size=10, shuffle=True)
testset = torch.utils.data.DataLoader(test, batch_size=10, shuffle=True)

100.0%


Extracting FashionMNIST\raw\train-images-idx3-ubyte.gz to FashionMNIST\raw


100.0%


Extracting FashionMNIST\raw\train-labels-idx1-ubyte.gz to FashionMNIST\raw




100.0%


Extracting FashionMNIST\raw\t10k-images-idx3-ubyte.gz to FashionMNIST\raw



100.0%

Extracting FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to FashionMNIST\raw



In [3]:
# Creating a class with a forward NN with 3 hidden layers with 64 output size

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 10)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.log_softmax(self.fc4(x), dim=1)
        
        return x

net = Net()
print(net)

Net(
  (fc1): Linear(in_features=784, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
)


In [4]:
# Using Adam optimizer to optimize weights of the Neural Network

optimizer = optim.Adam(net.parameters(), lr =0.001)

EPOCHS = 3

for epoch in range(EPOCHS):
    for data in trainset:
        X, y = data
        net.zero_grad()
        X = X.view(-1, 28*28) #10 x 1 x 28 x 28 = 7840, so the first dimension is computed as 7840 / (28 x 28) = 10.
        output = net(X)
        loss = F.nll_loss(output, y)
        loss.backward()
        optimizer.step()
    print(loss)

tensor(0.2746, grad_fn=<NllLossBackward0>)
tensor(0.3943, grad_fn=<NllLossBackward0>)
tensor(0.0477, grad_fn=<NllLossBackward0>)


In [5]:
# Calculating Accuracy

correct = 0
total = 0

with torch.no_grad(): # By default, PyTorch tracks the gradients of all tensor operations
     for data in testset:
            X, y = data
            output = net(X.view(-1, 784))
            for idx, i in enumerate(output):
                if torch.argmax(i) == y[idx]:
                    correct += 1
                total +=1
                
print("Accuracy: ", round(correct/total, 3))

Accuracy:  0.858
